In [5]:

import torch
import torch.nn as nn
#from vision_transformer import MyViT
from model import VisionTransformer
import frame_generator
import numpy as np
import os
import random


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
encoded_arrays = np.load("frames_data.npy")

In [3]:
chw = [1, 240, 304]
batch_size = 30
model = VisionTransformer(chw).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

c:\Users\paolo\OneDrive\Desktop\Progetti VSCode\Progetto_computer_vision\model.py:71: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.pos_embed = nn.Parameter(torch.tensor(get_positional_embeddings(n_patches**2, self.hidden_d)))


In [6]:
num_epochs = 3

for epoch in range(num_epochs):

 input_directory = "train_a/detection_dataset_duration_60s_ratio_1.0/train"
 i = 0

 print("EPOCH NUMBER ", epoch)

 all_files = [f for f in os.listdir(input_directory) if f.endswith(".npy")]
 random.shuffle(all_files)


 for filename in all_files:
    if filename.endswith(".npy"):
        file_path = os.path.join(input_directory, filename)
        gen1_bboxes = np.load(file_path)
        pedestrian_bboxes = 0
        vehicle_bboxes = 0
        for bbox in gen1_bboxes:    
            if bbox[5] == 1:
             pedestrian_bboxes += 1
            if bbox[5] == 0:
                vehicle_bboxes += 1
        if pedestrian_bboxes > 10 and vehicle_bboxes > 10:        #controllo che ci siano abbastanza occorenze per entrambe le classi
            print(pedestrian_bboxes, vehicle_bboxes)
            frames = encoded_arrays[i]
            classes = frame_generator.get_class(file_path, frames)
            i += 1
        
            total_correct = 0
            total_samples = 0
            total_loss = 0

            memory = torch.zeros(1 ,256, 512).to(device)

        
            for frame, target in zip(frames, classes):
            
            
            
                if(len(target) == 0):
                 with torch.no_grad():
                    input = torch.Tensor(frame[2]).to(device)
                    optimizer.zero_grad()
                    output, memory  = model(input.unsqueeze(0).unsqueeze(0), memory)

                if(len(target) != 0):
                 with torch.enable_grad():
                    input = torch.Tensor(frame[2]).to(device)
                    target = torch.tensor(target, dtype=torch.float32).to(device)
                    optimizer.zero_grad()
                    output, memory = model(input.unsqueeze(0).unsqueeze(0), memory)
                    loss = criterion(output, target)
                    loss.backward(retain_graph=True)
                    optimizer.step()
                    total_samples += 1
                    total_loss += loss

                    if(output >= 0.5 and target==1):
                        total_correct += 1

                    if(output < 0.5 and target==0):
                        total_correct +=1


            accuracy = total_correct/total_samples        
            average_loss = total_loss / total_samples
            print(f"Average Loss and accuracy for {filename}: {average_loss},  {accuracy*100}%")

EPOCH NUMBER  0
173 588
Average Loss and accuracy for 18-03-29_12-30-10_122500000_182500000_bbox.npy: 0.6425217986106873,  78.16901408450704%
79 47
Average Loss and accuracy for 17-10-12_17-56-37_793500000_853500000_bbox.npy: 0.8221017122268677,  37.254901960784316%
56 31
Average Loss and accuracy for 17-03-30_12-53-58_183500000_243500000_bbox.npy: 0.7337310910224915,  50.0%
35 126
Average Loss and accuracy for 17-04-11_12-00-08_500000_60500000_bbox.npy: 0.5290164351463318,  90.21739130434783%
11 87
Average Loss and accuracy for 17-10-09_16-14-57_488500000_548500000_bbox.npy: 0.21463121473789215,  100.0%
94 127
Average Loss and accuracy for 17-10-12_17-47-19_305500000_365500000_bbox.npy: 0.5240839719772339,  83.33333333333334%
163 92
Average Loss and accuracy for 17-10-09_16-14-57_500000_60500000_bbox.npy: 0.8012524843215942,  52.54237288135594%
45 51
Average Loss and accuracy for 17-10-05_13-55-39_610500000_670500000_bbox.npy: 0.6853819489479065,  56.41025641025641%
200 610
Average Lo

In [8]:
torch.save(model.state_dict(), 'pesi.pth')